In [83]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
#import matplotlib.pyplot as plt
from sklearn import metrics
import numpy as np


currency_list = ['USDCHF10080',
                 #'GBPUSD10080', 'EURUSD10080', 'USDJPY10080', 'USDCAD10080', 'AUDUSD10080', 'NZDUSD10080',
                 #'GBPCHF10080', 'EURCHF10080', 'CHFJPY10080', 'CADCHF10080', 'AUDCHF10080', 'NZDCHF10080', 'EURGBP10080',
                 #'GBPJPY10080', 'GBPCAD10080', 'GBPAUD10080', 'EURJPY10080', 'EURCAD10080', 'EURAUD10080', 'EURNZD10080',
                #'CADJPY10080', 'AUDJPY10080', 'NZDJPY10080', 'AUDCAD10080', 'NZDCAD10080', 'AUDNZD10080'
                ]





In [84]:
# #loops through all currency, identify timeframes not in nzdcad and save them in global_available_timeframes list

headers = ['date_start', 'ignore', 'open', 'high', 'low', 'close', 'volume'  ]

shortest_row_currency = pd.read_csv('recent_data/' + 'NZDCAD10080' + '.csv', names=headers  )  

#this reverses the rows so as to have the recent timeframe at the top
shortest_row_currency = shortest_row_currency.sort_index(axis=0,ascending=False)



shortest_row_currency_timeframes = shortest_row_currency['date_start']



shortest_row_currency_timeframes = shortest_row_currency_timeframes.values.tolist()




global_unavailable_timeframes = []
for x in currency_list:
    
    
    current_currency_data = pd.read_csv('recent_data/' + x + '.csv', names=headers  )  
    
    current_currency_data_timeframes = current_currency_data['date_start']
   
    current_currency_data_timeframes = current_currency_data_timeframes.values.tolist()
   
    for y in shortest_row_currency_timeframes:
        
            if y not in current_currency_data_timeframes :
                
                global_unavailable_timeframes.append(y)
                

# this get the unique time frames because time frames could have been duplicated from different currencies 
global_unavailable_timeframes = np.unique(global_unavailable_timeframes)   


print(len(global_unavailable_timeframes))   
print(len(shortest_row_currency_timeframes))


0
728


In [85]:
#removes the identified time frame nzdcad timeframe list to ensure that all timeframe in nzdcad list have 
#data in other currencies

globally_available_time_frame=[]

for x in global_unavailable_timeframes:
    
    shortest_row_currency_timeframes.remove(x)
    
globally_available_time_frame = shortest_row_currency_timeframes

print(len(globally_available_time_frame))

728


In [86]:
# loop through all currency pairs, get the needed rows from them and assemble all rows to form a combine_data_dataframe

combine_data_dataframe = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )

for x in currency_list :
    
    

    current_currency_data = pd.read_csv('recent_data/' + x + '.csv', names=headers  ) 
    
    # this sort the rows, so as to have the most recent time frame at the top
    current_currency_data = current_currency_data.sort_index(axis=0,ascending=False)
    print(current_currency_data)
    current_currency_volume = []
    
    current_currency_class = []
    
    q = 0

    for y in globally_available_time_frame:

        for index, row in current_currency_data.iterrows():
           
            
            if y  == row.date_start:            
                        
                current_currency_volume.append(row.volume)            
                
                # condition when the curreny movement is a buy

                pips_corrector = 100000

                if x.endswith('jpy'):# use this to correct multiplier of jpy pairs
                    pips_corrector = 1000

                if row.open < row.close:

                    trend = 1


                    #classifying the risk level


                    if (row.open - row.low) == 0 : # no risk because open is equal to low

                        risk = 6

                    else : 

                        if (row.open - row.low)> (row.high - row.open):

                            risk = 1

                        if  (row.high - row.open) > (row.open - row.low):

                            risk = 2


                        if (row.high - row.open) /(row.open - row.low) >2 :

                            risk = 3

                        if (row.high - row.open) /(row.open - row.low) >4 :

                            risk = 4


                        if (row.high - row.open) /(row.open - row.low) >8 :

                            risk = 5

                        if (row.high - row.open) /(row.open - row.low) >16 :

                            risk = 6  

                    #classifying reward level

                    if (row.close - row.open)*pips_corrector > 0 :

                        reward = 1

                    if (row.close - row.open)*pips_corrector >200 :

                        reward = 2

                    if (row.close - row.open)*pips_corrector > 400 :

                        reward = 3


                    if (row.close - row.open)*pips_corrector > 800 :

                        reward = 4

                    if (row.close - row.open)*pips_corrector > 1600 :

                        reward = 5


                    if (row.close - row.open)*pips_corrector > 3200 :

                        reward = 6  




                # this is for the condition when the movement is a sell
                else: 

                    trend = -1

                    #classifying the risk level


                    if (row.high - row.open) == 0 : # no risk because open is equal to high

                        risk = 6

                    else : 

                        if (row.high - row.open)> (row.open - row.low):

                            risk = 1

                        if  (row.open - row.low) > (row.high - row.open):

                            risk = 2


                        if (row.open - row.low) /(row.high - row.open) >2 :

                            risk = 3

                        if (row.open - row.low) /(row.high - row.open) >4 :

                            risk = 4

                        if (row.open - row.low) /(row.high - row.open) >8 :

                            risk = 5

                        if (row.open - row.low) /(row.high - row.open) >16 :

                            risk = 6 


                     #classifying reward level

                    if (row.open - row.close)*pips_corrector > 0 :

                        reward = 1

                    if (row.open - row.close)*pips_corrector > 200 :

                        reward = 2

                    if (row.open - row.close)*pips_corrector > 400 :

                        reward = 3        

                    if (row.open - row.close)*pips_corrector > 800 :

                        reward = 4    

                    if (row.open - row.close)*pips_corrector > 1600 :

                        reward = 5 

                    if (row.open - row.close)*pips_corrector > 3200 :

                        reward = 6        

            current_currency_class.append(trend*risk*reward)
            
        

            #attach the classification score value to the dataframe and train a model with the data
            #print(index)
    print(current_currency_class)

            #combine_data_dataframe[x+'_class'] = current_currency_class


            
                        

    # this function takes a list and returns with a list of 5 list that is the data for the previous row
    
    def add_5_columns(real_column) :
        
        current_currency_column_add_on = [real_column, [], [], [], [], []]
        
        g = 1 
        
        while g < 6:
            
            current_currency_column_add_on[g] = current_currency_column_add_on[g-1].copy()
            current_currency_column_add_on[g].pop(0)
            current_currency_column_add_on[g].append(0)    
            g = g+1
            
        current_currency_column_add_on.pop(0)
        return current_currency_column_add_on
            
            
           

    combine_data_dataframe[ x +'_volume'] = current_currency_volume
    
    #volume_columns = add_5_columns(current_currency_volume)
    
    for h in volume_columns:
        mini_df = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )
        mini_df[''] = h   
        combine_data_dataframe = pd.concat([ combine_data_dataframe, mini_df], axis=1)
    
   
    
    
    combine_data_dataframe[ x +'_open'] = current_currency_open
    
    #open_columns = add_5_columns(current_currency_open)
    
    for h in open_columns:
        mini_df = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )
        mini_df[''] = h   
        combine_data_dataframe = pd.concat([ combine_data_dataframe, mini_df], axis=1)
    
    
    
    combine_data_dataframe[ x +'_high'] = current_currency_high   
    
    #high_columns = add_5_columns(current_currency_high)
    
    for h in high_columns:
        mini_df = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )
        mini_df[''] = h   
        combine_data_dataframe = pd.concat([ combine_data_dataframe, mini_df], axis=1)
    
    
    
    combine_data_dataframe[ x +'_low'] = current_currency_low
    
    #low_columns = add_5_columns(current_currency_low)
    
    for h in low_columns:
        mini_df = pd.DataFrame(data={}, index = pd.Series(range(0,len(globally_available_time_frame))), )
        mini_df[''] = h   
        combine_data_dataframe = pd.concat([ combine_data_dataframe, mini_df], axis=1)
    
    
    
    
    combine_data_dataframe[ x +'_close'] = current_currency_close
    
   
    
    print(x)


      date_start ignore     open     high      low    close  volume
2587  2021.08.29  00:00  0.91084  0.91891  0.91009  0.91415  158583
2586  2021.08.22  00:00  0.91594  0.91997  0.91061  0.91105  150969
2585  2021.08.15  00:00  0.91494  0.92067  0.90994  0.91689  143626
2584  2021.08.08  00:00  0.91364  0.92421  0.91349  0.91523  128138
2583  2021.08.01  00:00  0.90439  0.91560  0.90183  0.91499  154705
...          ...    ...      ...      ...      ...      ...     ...
4     1971.01.31  00:00  4.29680  4.29690  4.29650  4.29690       5
3     1971.01.24  00:00  4.30050  4.30050  4.29630  4.29630       5
2     1971.01.17  00:00  4.30290  4.30290  4.29960  4.29960       5
1     1971.01.10  00:00  4.31020  4.31020  4.30760  4.30760       5
0     1971.01.03  00:00  4.31800  4.31800  4.31030  4.31090       5

[2588 rows x 7 columns]
[-9]


ValueError: Length of values (708) does not match length of index (728)

In [ ]:
combine_data_dataframe

In [ ]:
#create and add the timeframe column to the combine_data_data_frame to be able to identify each rows

date_series = pd.Series(globally_available_time_frame)

# the code below convert it to panda date time object
date_series = pd.to_datetime(date_series)


#turning date_series to a data frame so as to be able to add it as the first column 
date_series_dataframe =  pd.DataFrame(data={}, index = pd.Series(range(0, len(globally_available_time_frame))), )
date_series_dataframe['date_start'] = date_series

#the two dataframes are combined along the y axis to form one dataframe with all information and date
combine_data_dataframe = pd.concat([ date_series_dataframe, combine_data_dataframe], axis=1)

combine_data_dataframe

In [ ]:
# copies the data frame so as to isolate the first 10 rows for prediction
combine_top_10_rows_dataframe = combine_data_dataframe.copy(deep=True)


combine_top_10_rows_dataframe = combine_top_10_rows_dataframe.head(25)

#saves the first 10 rows to excel

combine_top_10_rows_dataframe.to_excel("files/combine_top_10_rows_dataframe.xlsx")



combine_data_dataframe = combine_data_dataframe.head( len(globally_available_time_frame) - 5 )

combine_data_dataframe.to_excel("files/combine_data_dataframe.xlsx") 
